In [1]:
import pandas as pd
import os
import datetime
import scipy
import pickle

In [2]:
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure
def create_flight_time(df, col):    
    liste = []
    for cols in df[col]:
        if pd.isnull(cols):
            liste.append(np.nan)
        elif float(cols) == 2400:
            cols = datetime.time(0,0)
            liste.append(cols)
        else:
            cols = format_heure(cols)
            liste.append(cols)
    return pd.Series(liste)

def concatenate():

    # load csv file containing the airport information
    airport_df = pd.read_csv("./airports.csv")

    #load csv file containing the flight delay information. this will be the main dataframe
    flights_df =  pd.read_csv('/home/chun/Desktop/gpr_testing/flight/flights.csv')
    main_df = flights_df
    # combine these, so that the main dataframe has latitude longitude information
    airport_df = airport_df.set_index("IATA_CODE")
    for loc in ['ORIGIN','DESTINATION']:
        for dir in ['LATITUDE','LONGITUDE']:
            main_df[loc+'_'+dir] =(airport_df[dir][flights_df[loc+"_AIRPORT"][:]]).reset_index()[[dir]]
#     main_df.to_csv("{}.txt".format('processed'))

    main_df['SCHEDULED_DEPARTURE'] = create_flight_time(main_df, 'SCHEDULED_DEPARTURE')
    main_df['SCHEDULED_HOUR']=pd.to_datetime(main_df['SCHEDULED_DEPARTURE'], format='%H:%M:%S').dt.hour
    airline_dummy = pd.get_dummies(main_df['AIRLINE'])
    main_df = main_df.merge(airline_dummy, left_index=True, right_index=True)
#     variables_to_remove = ['YEAR', 'DAY', 'AIRLINE', 'FLIGHT_NUMBER',
#        'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
#        'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'TAXI_OUT',
#        'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 
#        'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
#        'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
#        'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
#        'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
    variables_to_remove = ['YEAR', 'DAY', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
    main_df.drop(variables_to_remove, axis = 1, inplace = True)
    main_df.dropna(inplace = True)

    return main_df


In [3]:
# if __name__ == "__main__":
main_df = concatenate()
# pickle.dump(main_df, open('processed_airport.p', 'wb'))
# main_df.to_csv("{}.txt".format('processed_airport'))

/home/chun/py/py3env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
# tmp = main_df.head(5)
# tmp.shape
main_df['ori_des'] = main_df.apply(lambda x: (x['ORIGIN_AIRPORT']+'_'+x['DESTINATION_AIRPORT']), axis=1)

print(main_df.shape)
main_df.head(5)
# tmp

In [ ]:
pickle.dump(main_df, open('processed_airport_cross.p', 'wb'))

In [ ]:
main_df2 = pd.get_dummies(main_df, columns=['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'ori_des'])
print(main_df2.shape)

In [ ]:
pickle.dump(main_df, open('processed_airport_cross_dummy.p', 'wb'))

In [ ]:

main_df2.shape

In [ ]:
main_df2.to_csv("{}.txt".format('processed_airport2'))